### Experiment settings

In [12]:
from private_billing.server import TCPAddress

edge_address = TCPAddress("0.0.0.0", 5555)

In [13]:
import zmq
from private_billing.messages import Message
from private_billing.server import PickleEncoder

ctxt = zmq.Context()
sock = ctxt.socket(zmq.REQ)


def send(msg: Message, target: TCPAddress):
    with sock.connect(str(target)):
        enc = PickleEncoder.encode(msg)
        sock.send(enc)
        repl = sock.recv()
    return PickleEncoder.decode(repl)

### Start bootstrap

In [19]:
from src.experiment.experiment import BootStrapMessage

def bootstrap():
    msg = BootStrapMessage(None)
    resp = send(msg, edge_address)
    print(resp)
bootstrap()

### Start round

In [22]:
from private_billing.messages import ContextMessage
from private_billing.core import CycleContext, vector

def send_context(cycle):
    cyc_len = 744
    cyc = CycleContext(
        cycle,
        cyc_len,
        vector.new(cyc_len, 0.21),
        vector.new(cyc_len, 0.05),
        vector.new(cyc_len, 0.11),
    )
    msg = ContextMessage(None, cyc)
    send(msg, edge_address)

### Get telemetry data

In [23]:
from pathlib import Path
from src.experiment.telemetry import GetTelemetryMessage, TelemetryMessage

def get_telemetry(cycle_id):
    msg = GetTelemetryMessage(None, cycle_id)
    resp: TelemetryMessage = send(msg, edge_address)
    entries = resp.value
    
    if cycle_id == -1:
        nr_cores = len(entries)
    else:
        nr_cores = (len(entries) - 1) // 2
        
    fname = Path(f"telemetry-nr_cores_{nr_cores}_round_{cycle_id}.txt")
    
    with fname.open('w') as fp:        
        fp.writelines([str(entry) + "\n" for entry in entries])
    

In [24]:
# run experiment
import time

for i in range(10):
    send_context(i)
    time.sleep(20)
    get_telemetry(i)